In [4]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.72.0
    Uninstalling openai-1.72.0:
      Successfully uninstalled openai-1.72.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
/content/drive/MyDrive/news_data


In [5]:
import openai
import pandas as pd
import json
import os
from tqdm import tqdm
from google.colab import userdata

client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

top_50_tickers = ['AAPL', 'MSFT', 'NVDA', 'GOOG', 'GOOGL', 'AMZN', 'META',
                  'AVGO', 'TSLA', 'WMT', 'LLY', 'JPM', 'V', 'UNH', 'MA',
                  'XOM', 'COST', 'NFLX', 'PG', 'ORCL', 'JNJ', 'HD', 'ABBV',
                  'KO', 'TMUS', 'BAC', 'PM', 'CRM', 'CVX', 'PLTR', 'CSCO',
                  'MCD', 'IBM', 'ABT', 'LIN', 'WFC', 'GE', 'T', 'MRK',
                  'PEP', 'VZ', 'AXP', 'ACN', 'MS', 'ISRG', 'RTX', 'NOW',
                  'TMO', 'INTU', 'BX']

prompt_template = '''
              You are an expert in financial news and stock trading.

              You are given a news headline, a summary, and an automated sentiment label for a company called {company}.

              Please do the following:
              1. evidence: Extract important phrases from the headline and summary that support the sentiment. Output as a Python list.
              2. stock_movement: Given the sentiment and the snippet, predict the likely short-term stock movement of {company}. Output only "Up" or "Down".
              3. explanation: Briefly explain your reasoning behind the movement in 2–4 sentences.

              Headline: "{title}"

              Summary: "{summary}"

              Sentiment label: "{sentiment}"

              Return your output in JSON format with keys: evidence, stock_movement, explanation.

              '''

def analyze_news_article(title, summary, sentiment, company):
    prompt = prompt_template.format(title=title, summary=summary, sentiment=sentiment, company=company)

    try:
      response = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
              {"role": "system", "content": "You are a stock market expert"},
              {"role": "user", "content": prompt}
          ],
          temperature=0,
          max_tokens=300
      )
      return json.loads(response.choices[0].message.content.strip())
    except Exception as e:
      return {
            "evidence": [],
            "stock_movement": "Unknown",
            "explanation": f"Failed due to error: {e}"
            }


In [8]:
for ticker in tqdm(top_50_tickers):
    input_file_name = f"{ticker}_news.csv"
    output_file_name = f"{ticker}_llm_analysis.csv"
    input_file_path = f"/content/drive/MyDrive/news_data/{input_file_name}"
    output_file_path = f"/content/drive/MyDrive/news_data/{output_file_name}"

    if not os.path.exists(input_file_path):
        print(f" Missing input for {ticker}, skipping.")
        continue

    if os.path.exists(output_file_path):
        # print(f"Already processed {ticker}, skipping.")
        os.remove(output_file_path)
        continue

    df = pd.read_csv(input_file_path)

    df["title"] = df["title"].fillna("")
    df["summary"] = df["summary"].fillna("")
    df["overall_sentiment_label"] = df["overall_sentiment_label"].fillna("Neutral")

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"{ticker}"):
        title = row["title"]
        summary = row["summary"]
        sentiment = row["overall_sentiment_label"]
        company = row["ticker"]
        result = analyze_news_article(title, summary, sentiment, company)
        results.append(result)

    df["llm_evidence"] = [r["evidence"] for r in results]
    df["llm_predicted_movement"] = [r["stock_movement"] for r in results]
    df["llm_explanation"] = [r["explanation"] for r in results]

    df.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_name}")

  2%|▏         | 1/50 [08:15<6:44:46, 495.64s/it]

Saved: AAPL_llm_analysis.csv



  4%|▍         | 2/50 [19:43<8:07:11, 608.98s/it]

Saved: MSFT_llm_analysis.csv



  6%|▌         | 3/50 [36:27<10:18:18, 789.33s/it]

...

Saved: TMO_llm_analysis.csv



 98%|█████████▊| 49/50 [2:26:13<00:48, 48.66s/it]

Saved: INTU_llm_analysis.csv



100%|██████████| 50/50 [2:27:06<00:00, 176.53s/it]

Saved: BX_llm_analysis.csv
